In [1]:
!pip install python-chess
!pip install psutil
!pip install google-cloud-storage

import numpy as np
import pandas as pd
import glob
import os
import chess


In [4]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gleaming-modem-400906-3780658b0f47.json"
from google.cloud import storage

def download_from_bucket(blob_name, path_to_save_to, bucket_name):
    client = storage.Client()

    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(path_to_save_to)
    print(f"File {blob_name} downloaded to {path_to_save_to}.")

# Use the function to download your file:
download_from_bucket("final_4M_bits.npy", "/content/final_4M_bits.npy", "chess_data_all")
download_from_bucket("final_4M_labels.npy", "/content/final_4M_labels.npy", "chess_data_all")


File final_4M_bits.npy downloaded to /content/final_4M_bits.npy.
File final_4M_labels.npy downloaded to /content/final_4M_labels.npy.


In [2]:
bitboard_dataset=np.load("/content/final_4M_bits.npy")
labels=np.load("/content/final_4M_labels.npy")

In [3]:
bitboard_dataset2=bitboard_dataset[:2000000]
labels2=labels[:2000000]


In [ ]:
# print(bitboard_dataset2.shape ,bitboard_dataset2[0])
# print(labels2.shape ,labels2[0])

In [4]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Concatenate
from tensorflow.keras.activations import relu
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D


# ... Assuming bitboard_dataset and labels are defined ...

bitboard_dataset, labels = shuffle(bitboard_dataset2, labels2)
X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset2, labels2, test_size=0.1)


# Input layer
input_layer_2d = layers.Input(shape=(16,8, 8))  # for 2D convolutions
input_layer_3d = layers.Reshape((16, 8, 8, 1))(input_layer_2d)  # for 3D convolutions


In [5]:
from tensorflow.keras.layers import BatchNormalization, Dropout, Activation

# Branch 1: Depthwise Convolution
branch1 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(input_layer_2d)
branch1 = BatchNormalization()(branch1)
branch1 = Dropout(0.1)(branch1)


branch4 = Conv2D(filters=3, kernel_size=(8, 8), activation=relu)(branch1)
branch4 = BatchNormalization()(branch4)
branch4 = Dropout(0.1)(branch4)
# branch4 = Conv2D(filters=5, kernel_size=(2, 2), activation=relu)(branch4)

# Branch 2: Conv3D
branch2 = Conv3D(filters=16, kernel_size=(5, 5, 5), padding='valid', activation=relu)(input_layer_3d)
branch2 = BatchNormalization()(branch2)
branch2 = Dropout(0.1)(branch2)
branch2 = Conv3D(filters=8, kernel_size=(3, 3, 3), padding='valid', activation=relu)(branch2)
branch2 = BatchNormalization()(branch2)



# Branch 3: Conv3 (Only to the 15th channel)
branch3_input = tf.gather(input_layer_2d, [13,14], axis=3)  # Extract the 15th channel
branch3 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(branch3_input)
branch3 = Conv2D(filters=10, kernel_size=(8,8), activation=relu)(branch3)
branch3 = Conv2D(filters=2, kernel_size=(1, 1), activation=relu)(branch3)
branch3 = BatchNormalization()(branch3)
branch3 = Dropout(0.1)(branch3)


branch1 = Flatten()(branch1)
branch2 = Flatten()(branch2)  # Flatten branch2
branch4 = Flatten()(branch4)
branch3 = Flatten()(branch3)  # Flatten branch3

concat = Concatenate()([branch1,branch4, branch2, branch3])

# Dense layers
dense1 = Dense(2000)(concat)
dense1 = BatchNormalization()(dense1)
dense1 = Activation('relu')(dense1)
dense1 = Dropout(0.25)(dense1)

dense2 = Dense(1000)(dense1)
dense2 = BatchNormalization()(dense2)
dense2 = Activation('relu')(dense2)
dense2 = Dropout(0.25)(dense2)

dense3 = Dense(500)(dense2)
dense3 = BatchNormalization()(dense3)
dense3 = Activation('relu')(dense3)
dense3 = Dropout(0.25)(dense3)

dense4 = Dense(100)(dense3)
dense4 = BatchNormalization()(dense4)
dense4 = Activation('relu')(dense4)
dense4 = Dropout(0.25)(dense4)

output_layer = Dense(1)(dense4)

# Create the model
model = Model(inputs=input_layer_2d, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=500, validation_split=0.1)

# Evaluate the model
loss, mae= model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Mean Absolute Error:", mae)

Epoch 1/200
3240/3240 [==============================] - 55s 14ms/step - loss: 7172.2217 - mae: 33.4682 - val_loss: 6394.1147 - val_mae: 31.1108
Epoch 2/200
3240/3240 [==============================] - 44s 13ms/step - loss: 6338.1045 - mae: 32.1346 - val_loss: 5914.1353 - val_mae: 30.4273
Epoch 3/200
3240/3240 [==============================] - 44s 14ms/step - loss: 5778.7036 - mae: 31.2157 - val_loss: 5433.6616 - val_mae: 29.0331
Epoch 4/200
3240/3240 [==============================] - 44s 14ms/step - loss: 5224.9761 - mae: 30.2828 - val_loss: 5115.8774 - val_mae: 28.7749
Epoch 5/200
3240/3240 [==============================] - 44s 14ms/step - loss: 4709.9570 - mae: 29.3487 - val_loss: 4802.2246 - val_mae: 27.6896
Epoch 6/200
3240/3240 [==============================] - 44s 14ms/step - loss: 4343.1631 - mae: 28.5800 - val_loss: 4978.7168 - val_mae: 28.3240
Epoch 7/200
3240/3240 [==============================] - 44s 14ms/step - loss: 4004.6963 - mae: 27.8697 - val_loss: 4648.5146 - va

In [ ]:
model.save("model.h5")
